In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

In [6]:
# ========================================
# 경로 설정 (여기를 수정하세요)
# ========================================
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/기계학습/7주차 강의/cifar10_data'  # CIFAR-10 데이터를 저장할 경로

# 데이터 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)

# --- CIFAR-10 로드 (자동으로 캐싱) ---
# Keras는 기본적으로 ~/.keras/datasets에 저장하지만,
# 명시적으로 경로를 지정하려면 아래와 같이 처리
cifar10_path = os.path.join(DATA_DIR,'cifar-10-batches-py')

if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")

# 데이터 로드 (없으면 자동 다운로드 후 캐싱)
(X_train, y_train),(X_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {X_train.shape}")
print(f"  - 테스트 데이터: {X_test.shape}\n")

# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

X_train, y_train = X_train[train_mask], y_train[train_mask]
X_test, y_test = X_test[test_mask], y_test[test_mask]

print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(X_train)}")
print(f"  - 테스트 샘플 수: {len(X_test)}\n")

# 라벨을 0~2로 다시 매핑
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[int(y)]for y in y_train])
y_test = np.array([label_map[int(y)]for y in y_test])

# 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0

데이터가 없습니다. /content/drive/MyDrive/Colab Notebooks/기계학습/7주차 강의/cifar10_data에 다운로드합니다...
✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)

선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



/tmp/ipython-input-857400948.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_train = np.array([label_map[int(y)]for y in y_train])
/tmp/ipython-input-857400948.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_test = np.array([label_map[int(y)]for y in y_test])


In [7]:
# 데이터 shape 확인
print(X_train.shape, X_test.shape)

(15000, 32, 32, 3) (3000, 32, 32, 3)


In [8]:
# CNN 모델 구성
#
model = models.Sequential([
    # 3x3 크기로 32개의 서로 다른 특징을 찾는다.  -> 단순
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    #  찾아낸 특징을 2x2 구역으로 나누고 그 안에서 가장 큰 특징만 골라낸다.
    layers.MaxPooling2D((2, 2)),
    # 3x3 크기로 64개의 서로 다른 특징을 찾는다.  -> 복합
    layers.Conv2D(64, (3, 3), activation='relu'),
    # 역시 2x2 구역으로 나누고 큰 특징만 골라낸다.
    layers.MaxPooling2D((2, 2)),
    # 마지막으로 3x3 크기로 128개의 서로 다른 특징을 찾는다. -> 추
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# 학습
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

Epoch 1/10
106/106 - 20s - 188ms/step - accuracy: 0.4919 - loss: 0.9661 - val_accuracy: 0.5987 - val_loss: 0.8689
Epoch 2/10
106/106 - 18s - 167ms/step - accuracy: 0.6148 - loss: 0.8082 - val_accuracy: 0.6327 - val_loss: 0.7980
Epoch 3/10
106/106 - 22s - 206ms/step - accuracy: 0.6551 - loss: 0.7480 - val_accuracy: 0.6533 - val_loss: 0.7630
Epoch 4/10
106/106 - 19s - 180ms/step - accuracy: 0.6914 - loss: 0.6805 - val_accuracy: 0.7000 - val_loss: 0.6720
Epoch 5/10
106/106 - 22s - 205ms/step - accuracy: 0.7133 - loss: 0.6412 - val_accuracy: 0.7247 - val_loss: 0.6520
Epoch 6/10
106/106 - 19s - 178ms/step - accuracy: 0.7301 - loss: 0.6088 - val_accuracy: 0.7233 - val_loss: 0.6422
Epoch 7/10
106/106 - 18s - 166ms/step - accuracy: 0.7486 - loss: 0.5791 - val_accuracy: 0.7420 - val_loss: 0.6091
Epoch 8/10
106/106 - 17s - 165ms/step - accuracy: 0.7619 - loss: 0.5505 - val_accuracy: 0.7473 - val_loss: 0.5974
Epoch 9/10
106/106 - 20s - 187ms/step - accuracy: 0.7773 - loss: 0.5199 - val_accuracy: 

In [11]:
# 평가
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

94/94 - 1s - 12ms/step - accuracy: 0.7613 - loss: 0.5637

✅ Test Accuracy: 0.7613


In [12]:
# 예측 예시
predictions = model.predict(X_test[:5])
print("예측 결과:", np.argmax(predictions, axis=1))
print("실제 레이블:", y_test[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
예측 결과: [0 0 1 2 1]
실제 레이블: [0 0 1 2 1]
